In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
train = pd.read_csv('./data/train.csv', parse_dates=[0], index_col=0)

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10506 entries, 2007-05-29 to 2013-09-26
Data columns (total 11 columns):
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 984.9+ KB


In [4]:
train.groupby('Trap').WnvPresent.sum().sort_values(ascending=False).head()

Trap
T900    66
T115    41
T002    18
T138    16
T003    14
Name: WnvPresent, dtype: int64

In [5]:
train.groupby('Species').WnvPresent.value_counts()

Species                 WnvPresent
CULEX ERRATICUS         0                1
CULEX PIPIENS           0             2459
                        1              240
CULEX PIPIENS/RESTUANS  0             4490
                        1              262
CULEX RESTUANS          0             2691
                        1               49
CULEX SALINARIUS        0               86
CULEX TARSALIS          0                6
CULEX TERRITANS         0              222
Name: WnvPresent, dtype: int64

In [6]:
spray = pd.read_csv('./data/spray.csv')

In [7]:
spray.Date = spray.Date.map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [8]:
spray['year'] = spray.Date.map(lambda x: x.year)

In [9]:
spray.groupby('year').count()

,Date,Time,Latitude,Longitude
year,,,,
2011,2209,1625,2209,2209
2013,12626,12626,12626,12626


In [10]:
def combine_rows(df):
    place_table = df[[
        'Address', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy'
    ]].drop_duplicates().set_index(['Address'])

    totals = df.groupby([df.index, 'Trap', 'Address', 'Species']).NumMosquitos.sum()
    has_nile = df.groupby([df.index, 'Trap', 'Address', 'Species']).WnvPresent.sum().map(lambda x: int(x > 0))
    return pd.concat([totals, has_nile], axis=1).reset_index().merge(place_table.drop('Trap', axis=1), on='Address').set_index('Date')

In [11]:
#confused how i want to handle the df
def deal_w_stations(df2, weather):
    station1Lat = 41.995
    station1Lon = -87.933
    station2Lat = 41.786
    station2Lon = -87.752

    dist1 = abs((df2.Latitude - station1Lat)*1j + (df2.Longitude - station1Lon))
    dist2 = abs((df2.Latitude - station2Lat)*1j + (df2.Longitude - station2Lon))
    compare = dist1 < dist2
    weather_fields = weather.select_dtypes([float,int]).columns[1:]

    for field in weather_fields:
        df2[field] = df2[field + '_x']*compare + df2[field + '_y']*(1-compare)

    redundant_cols = [name for name in df2.columns if '_' in name]

    return df2.drop(redundant_cols, axis=1)

In [12]:
weather = pd.read_csv('./data/ps_weather_processed.csv', parse_dates=[0], index_col=0)\
                .assign(HCDiff = lambda df: df.Heat - df.Cool)\
                .apply(lambda col: col.replace('-',None).ffill().astype(int) if col.name in ['Sunrise', 'Sunset'] else col)\
                .drop(['Heat', 'Cool','isRain'], axis=1)


In [13]:
shift_dict_default = {
        'Tmin':7,
        'Tmax':7,
        'Tavg':7,
}

In [14]:
#used in conjunction with apply
def weatherShiftLogic(col, shift_dict=shift_dict_default):
    if col.name in shift_dict:
        return col.shift(shift_dict[col.name])
    else:
        return col.shift(21)

In [15]:
def merge_weather(df):
    weather = pd.read_csv('./data/ps_weather_processed.csv', parse_dates=[0], index_col=0)\
                .apply(lambda col: col.replace('-',None).ffill().astype(int) if col.name in ['Sunrise', 'Sunset'] else col)\
                .assign(HCDiff = lambda df: df.Heat - df.Cool, sundiff = lambda df: df.Sunset - df.Sunrise)\
                .drop(['Heat', 'Cool','isRain', 'Sunrise', 'Sunset'], axis=1)

    a = weather.query('Station==1').drop('Station', axis=1).select_dtypes([float,int]).apply(weatherShiftLogic)
    b = weather.query('Station==2').drop('Station', axis=1).select_dtypes([float,int]).apply(weatherShiftLogic)
    df = df.join([a, b]).dropna()
    return deal_w_stations(df, weather)

In [16]:
def adjust_columns(df2):
    relevant_species = ['CULEX PIPIENS', 'CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS']
    for species in relevant_species:
        df2[species] = df2.Species.map(lambda x: int(x==species))
    df2 = df2.assign(none=df2.Species.map(lambda x: int(x not in relevant_species )), month = df2.index.month, lat2 = df2.Latitude**2, lon2 = df2.Longitude**2, month2 = df2.index.month**2)
    df2 = df2.assign(year = df2.index.year, year2 = df2.index.year**2)
    df2 = df2.select_dtypes([int, float]).drop(['Block', 'AddressAccuracy'], axis=1)
    return df2

In [17]:
def naivestrap(x, y):
    positive_index = y.reset_index().query('WnvPresent==1').index
    indices = np.random.choice(positive_index, size=len(y)-y.sum())
    return x.copy().iloc[indices,:], y.copy().iloc[indices]

In [18]:
df2 = combine_rows(train)

In [19]:
df2 = merge_weather(df2)

In [20]:
df2 = adjust_columns(df2)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x = df2.drop(['WnvPresent','NumMosquitos'], axis=1)
y = df2.WnvPresent

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [23]:
xtrainint, ytrainint = naivestrap(xtrain, ytrain)
xtrain = pd.concat([xtrain, xtrainint])
ytrain = pd.concat([ytrain, ytrainint])

# Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [25]:
scaler = StandardScaler()

In [26]:
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [27]:
logreg = LogisticRegression(C=0.05, penalty='l2')
logreg.fit(xtrain, ytrain)
logreg.score(xtest, ytest)

0.6401887534408179

In [28]:
roc_auc_score(ytrain, [proba[1] for proba in logreg.predict_proba(xtrain)])

0.8024662223449527

In [29]:
roc_auc_score(ytest, [proba[1] for proba in logreg.predict_proba(xtest)])

0.8221222653854018

In [30]:
sorted(list(zip(x.columns, logreg.coef_[0])), key=lambda x: x[1])

[('StnPressure', -0.8233733522055148),
 ('none', -0.6326264379421478),
 ('DewPoint', -0.4881191770302534),
 ('Tmax', -0.3673802961813744),
 ('HCDiff', -0.3309446342440505),
 ('CULEX RESTUANS', -0.2640608270193328),
 ('Longitude', -0.18500047319669863),
 ('Latitude', -0.14413522147236302),
 ('ResultSpeed', -0.1372885619212409),
 ('lat2', -0.12165697266960587),
 ('PrecipTotal', -0.1121744678417587),
 ('AvgSpeed', 0.02256421770335946),
 ('ResultDir', 0.05947128514431633),
 ('month2', 0.08070350951111813),
 ('CULEX PIPIENS/RESTUANS', 0.10918062076637437),
 ('lon2', 0.19112092066474862),
 ('Tmin', 0.19886723798672853),
 ('year', 0.22357441626156563),
 ('year2', 0.24085678661897972),
 ('CULEX PIPIENS', 0.2901446528095992),
 ('Tavg', 0.43408160268776813),
 ('WetBulb', 0.6065708371781048),
 ('SeaLevel', 0.7170580543580929),
 ('sundiff', 0.9276829581077158),
 ('month', 1.3524285212304104)]

In [31]:
model = logreg

In [32]:
import seaborn as sns

In [33]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8475 entries, 2007-05-29 to 2013-09-26
Data columns (total 27 columns):
NumMosquitos              8475 non-null int64
WnvPresent                8475 non-null int64
Latitude                  8475 non-null float64
Longitude                 8475 non-null float64
Tmax                      8475 non-null float64
Tmin                      8475 non-null float64
Tavg                      8475 non-null float64
DewPoint                  8475 non-null float64
WetBulb                   8475 non-null float64
PrecipTotal               8475 non-null float64
StnPressure               8475 non-null float64
SeaLevel                  8475 non-null float64
ResultSpeed               8475 non-null float64
ResultDir                 8475 non-null float64
AvgSpeed                  8475 non-null float64
HCDiff                    8475 non-null float64
sundiff                   8475 non-null float64
CULEX PIPIENS             8475 non-null int64
CULEX PIPIENS/RES

In [34]:
df2.corr().WnvPresent

NumMosquitos              0.233532
WnvPresent                1.000000
Latitude                  0.030862
Longitude                -0.076732
Tmax                      0.043678
Tmin                      0.074797
Tavg                      0.064629
DewPoint                  0.081697
WetBulb                   0.104431
PrecipTotal              -0.029796
StnPressure              -0.000360
SeaLevel                  0.015840
ResultSpeed              -0.052423
ResultDir                 0.018955
AvgSpeed                 -0.060212
HCDiff                   -0.119001
sundiff                  -0.006183
CULEX PIPIENS             0.097868
CULEX PIPIENS/RESTUANS    0.019620
CULEX RESTUANS           -0.094047
none                     -0.046206
month                     0.097948
lat2                      0.030916
lon2                      0.076766
month2                    0.088755
year                      0.043038
year2                     0.043098
Name: WnvPresent, dtype: float64

In [35]:
test = pd.read_csv('./data/test.csv', parse_dates=[1], index_col=1)
test = merge_weather(test)
test = adjust_columns(test)
test = test.drop('Id', axis=1)

In [36]:
test = scaler.transform(test)

In [37]:
submission = pd.DataFrame([predictions[1] for predictions in model.predict_proba(test)], columns=['WnvPresent'])
submission = submission.reset_index().rename({'index':'Id'}, axis=1)
submission.loc[:,'Id'] = submission.Id + 1
submission = submission.set_index('Id')
submission.to_csv('submissionlog.csv')